In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc
import time
from fractions import Fraction
from collections import defaultdict
from unicodedata import normalize
from itertools import zip_longest
import os

This code scrapes several betting odds from Oddschecker.com, converts the odds to percentages and calculates predicted points for players in the next full gameweek in Fantasy Premier League according to the percentages. In addition to selenium, webdriver has to be installed also. Webdrivers run or drive a browser from inside of your code. Version of webdriver has to match the version of your browser.

Assisting and Goalscoring odds for players are usually available couple of days before the game, so this script is very likely to return empty file or a file containing a lot of missing values if there are still several days until the first game of the gameweek.

Added new functionalities to the code (Predicted_Manager_Points and Predicted_Player_Points) to better match a player from the Oddschecker website with the correct player from the FPL API. Additionally, the code has been improved to allow handling of game weeks where a team may play more than one match.

In [3]:
url = "https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch fixtures: {response.status_code}")
fixtures = response.json()

In [4]:
game_weeks = defaultdict(list)
for fixture in fixtures:
    game_weeks[fixture["event"]].append(fixture)
for event in sorted(game_weeks.keys()):
    if all(not fixture['finished'] for fixture in game_weeks[event]):
        next_gameweek = event
        break
    else:
        next_gameweek = None

In [5]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch teams: {response.status_code}")
data = response.json()
teams = data['teams']
team_id_to_name = {team['id']: team['name'] for team in teams}

In [6]:
TEAM_NAMES_ODDSCHECKER = {
    "Nott'm Forest": "Nottingham Forest",
    "Wolves": "Wolverhampton",
    "Spurs": "Tottenham",
    }

In [7]:
next_gw_fixtures = [fixture for fixture in fixtures if (fixture['event'] == next_gameweek) and (fixture['started'] == False)]
print("Next Gameweek Fixtures:")
print('')
teams_playing = defaultdict(int)
for fixture in next_gw_fixtures:
    teams_playing[TEAM_NAMES_ODDSCHECKER.get(team_id_to_name[fixture['team_h']], team_id_to_name[fixture['team_h']])] += 1
    teams_playing[TEAM_NAMES_ODDSCHECKER.get(team_id_to_name[fixture['team_a']], team_id_to_name[fixture['team_a']])] += 1
    print(team_id_to_name[fixture['team_h']], 'v.', team_id_to_name[fixture['team_a']])

Next Gameweek Fixtures:

Everton v. West Ham
Ipswich v. Nott'm Forest
Man City v. Brighton
Southampton v. Wolves
Bournemouth v. Brentford
Arsenal v. Chelsea
Fulham v. Spurs
Leicester v. Man Utd


In [ ]:
# Function to normalize and prepare names for comparison
def prepare_name(name):
    """
    Normalizes a name by converting to lowercase, removing accents, and splitting into tokens.
    """

    # Replace Scandinavian letters with their ASCII equivalents
    scandinavian_replacements = {
        'ø': 'o',
        'å': 'a',
        'æ': 'ae',
        'Ø': 'O',
        'Å': 'A',
        'Æ': 'AE',
    }
    for scandinavian_char, ascii_char in scandinavian_replacements.items():
        name = name.replace(scandinavian_char, ascii_char)


    # Normalize the name to handle accents and foreign characters
    normalized_name = normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
    cleaned_name = normalized_name.replace('-', ' ').replace("'", '')
    # Convert to lowercase and split into tokens
    return cleaned_name.lower().split()

In [ ]:
team_position = {team['id']: team['position'] for team in teams}
# Initialize player_dict to store lists of values for each key
player_dict = defaultdict(lambda: defaultdict(list))

# Map element_type to position
element_types = {et["id"]: et["singular_name_short"] for et in data["element_types"]}

for player in data["elements"]:
    player_name = player["first_name"] + " " + player["second_name"]
    nickname = player['web_name']
    nickname1 = player['web_name']
    nickname2 = player['web_name']
    index = nickname.find(".")
    while (index != -1):
        if index != len(nickname1) - 1:
            nickname1 = nickname1[:index] + ' ' + nickname1[index+1:]
            if nickname1.find(".") != -1:
                nickname1 = nickname1[index+1:]
                index = nickname1.find(".")
            index = nickname1.find(".")
        else:
            nickname1 = nickname1[:index]
            index = nickname1.find(".")

    index2 = nickname.find(".")
    while (index2 != -1):
        if index2 != len(nickname2) - 1:
            nickname2 = nickname2[index2+1:]
            index2 = nickname2.find(".")
        else:
            nickname2 = nickname2[:index2]
            index2 = nickname2.find(".")

    player_dict[player_name]['Nickname'] = nickname1 if nickname1 != None else nickname
    player_dict[player_name]['Nickname2'] = nickname2 if nickname2 != None else None 
    player_dict[player_name]['Position'] = element_types[player["element_type"]]
    player_dict[player_name]['Team'] = TEAM_NAMES_ODDSCHECKER.get(team_id_to_name[player["team"]], team_id_to_name[player["team"]])
    player_dict[player_name]['Chance of Playing'] = player['chance_of_playing_next_round'] / 100 if player['chance_of_playing_next_round'] else 1 if player['status'] in ('a', 'd') else 0
    
driver = uc.Chrome()  # Replace with the path to your WebDriver if needed
driver.get("https://www.oddschecker.com/football/english/premier-league/")
 
wait = WebDriverWait(driver, 10)

try:
    span_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/section/h2/span[2]')))
    # Click on the <span> element (Accessing outside UK pop-up)
    span_element.click()

except TimeoutException:
    print("Prompt for accessing outside UK did not pop up")
    
wait = WebDriverWait(driver, 3)
try:
    cookiebutton = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'CookieBannerAcceptButton_c1mxe743')))
    # Click on the accept cookies button
    cookiebutton.click()
except TimeoutException:
    print("Prompt for accepting Cookies did not pop up")

except ElementClickInterceptedException:
    try:
        wait = WebDriverWait(driver, 3)
        cookiebutton = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'CookieBannerAcceptButton_c1mxe743')))
        cookiebutton.click()
    except ElementClickInterceptedException:
        print("Prompt for accepting Cookies did not pop up")

wait = WebDriverWait(driver, 5)
try:
    close_ad = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'webpush-swal2-close')))
    # Click close ad button
    close_ad.click()
except TimeoutException:
    print('Ad did not pop up')

try:
    wait = WebDriverWait(driver, 3)
    matches_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Matches')]")))
    matches_button.click()
except Exception as e:
    print("Couldn't click Matches tab ", e)

matches_details = {}
for fixture in next_gw_fixtures:
    home_team_id = fixture['team_h']
    away_team_id = fixture['team_a']
    home_team_name = team_id_to_name.get(home_team_id, "Unknown Team")
    away_team_name = team_id_to_name.get(away_team_id, "Unknown Team")
    home_position = team_position.get(home_team_id, "Unknown Position")
    away_position = team_position.get(away_team_id, "Unknown Position")
    if abs(int(home_position) - int(away_position)) >= 5:
        if home_position > away_position:
            Underdog_Bonus = 'Home'
        else:
            Underdog_Bonus = 'Away'
    else:
        Underdog_Bonus = 'None'

    home_team = TEAM_NAMES_ODDSCHECKER.get(home_team_name, home_team_name)
    away_team = TEAM_NAMES_ODDSCHECKER.get(away_team_name, away_team_name)
    match_title = home_team + " v " + away_team

    # Find match link
    match_link = driver.find_element(By.XPATH, f"//a[@title='{match_title}'][@href]")
    href = match_link.get_attribute("href")

    matches_details[match_title] = {}
    matches_details[match_title]['home_team'] = home_team_name
    matches_details[match_title]['away_team'] = away_team_name
    matches_details[match_title]['Underdog Bonus'] = Underdog_Bonus
    matches_details[match_title]['Link'] = href
        
for match, details in matches_details.items():
    home_team_name = details.get('home_team', 'Unknown')
    away_team_name = details.get('away_team', 'Unknown')
    home_team = TEAM_NAMES_ODDSCHECKER.get(home_team_name, home_team_name)
    away_team = TEAM_NAMES_ODDSCHECKER.get(away_team_name, away_team_name)
    Underdog_Bonus = details.get('Underdog Bonus', 'None')
    link = details.get('Link', 'Link not found')
    
    try:
        driver.get(link)
    except Exception as e:
        print("Couldn't open link ", link, " ", e)

    wait = WebDriverWait(driver, 5)

    try:
        span_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/section/h2/span[2]')))
        # Click on the <span> element (Accessing outside UK pop-up)
        span_element.click()
    
    except TimeoutException:
        print("Prompt for accessing outside UK did not pop up")

    wait = WebDriverWait(driver, 3)
    try:
        cookiebutton = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'CookieBannerAcceptButton_c1mxe743')))
        # Click on the accept cookies button
        cookiebutton.click()
    except TimeoutException:
        print("Prompt for accepting Cookies did not pop up")

    wait = WebDriverWait(driver, 5)
    try:
        close_ad = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'webpush-swal2-close')))
        # Click close ad button
        close_ad.click()
    except TimeoutException:
        print('Ad did not pop up')
    
    try:
        win_market_header = driver.find_element(By.XPATH, "//h2[contains(text(), 'Win Market')]")
        # Expand the section if it's collapsed
        if win_market_header.get_attribute("aria-expanded") == "false":
            win_market_header.click()
            time.sleep(3)
        outcomes = driver.find_elements(By.XPATH, "//h2[contains(text(), 'Win Market')]/following-sibling::*[1]/*[1]/*[1]//p")
        for outcome in outcomes:
            odd = outcome.find_element(By.XPATH, "./following-sibling::button")
            if outcome.get_attribute("innerText") == home_team:
                home_win = odd.get_attribute("innerText")
            elif outcome.get_attribute("innerText") == away_team:
                away_win = odd.get_attribute("innerText")
            else:
                draw = odd.get_attribute("innerText")

    except Exception as e:
        home_win = 0
        away_win = 0
        draw = 0
        print("Could not find Win Market")

    for player in player_dict:
        if player_dict[player]['Position'] == 'MNG':
            if player_dict[player]['Team'] == home_team:
                player_dict[player]['Win Probability'].append(1/(float(Fraction(home_win)) + 1))
                player_dict[player]['Draw Probability'].append(1/(float(Fraction(draw)) + 1))
                if Underdog_Bonus == 'Home':
                    player_dict[player]['Manager Bonus'].append('True')
                else: 
                    player_dict[player]['Manager Bonus'].append('False')
                    
            if player_dict[player]['Team'] == away_team:
                player_dict[player]['Win Probability'].append(1/(float(Fraction(away_win)) + 1))
                player_dict[player]['Draw Probability'].append(1/(float(Fraction(draw)) + 1))
                if Underdog_Bonus == 'Away':
                    player_dict[player]['Manager Bonus'].append('True')
                else:
                    player_dict[player]['Manager Bonus'].append('False')
                
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'Player Assists')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand
            
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'Player Assists')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on Player Assists ")

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'Player Assists')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:
            try:
                name_text = player.get_attribute("innerText")
                if "0.5" in name_text:
                    name = name_text.replace('Over 0.5', '').strip()
           
                    # Get the odds element (button following the player name)
                    odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                    odd = odd_element.get_attribute("innerText").strip()

                    for p in player_dict:
                        # Prepare the player name for comparison
                        player_tokens = prepare_name(p)
                        webname_tokens = prepare_name(name)
                        matched_name = None

                        # Check if all tokens in one name exist in the other
                        if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                            matched_name = p
                            break

                    # Add the odds to the player's dictionary
                    if matched_name:

                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Assisting Over 0.5 Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Assisting Over 0.5 Probability"].append(0)
                            
                    else:
                        for p in player_dict:
                            # Prepare the player name for comparison
                            webname_tokens = prepare_name(name)
                            matched_name = None
                            nickname1 = player_dict[p]['Nickname']
                            nickname2 = player_dict[p]['Nickname2']
                            nickname_tokens = prepare_name(nickname2)

                            if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                                matched_name = p
                                break

                        if matched_name:
                            # Calculate and add the probability
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[matched_name]["Assisting Over 0.5 Probability"].append(probability)
                            else:
                                player_dict[matched_name]["Assisting Over 0.5 Probability"].append(0)

                        else:
                            player_dict[name]['Nickname'] = 'Unknown'
                            player_dict[name]['Position'] = 'Unknown'
                            player_dict[name]['Team'] = "Unknown"
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[name]["Assisting Over 0.5 Probability"].append(probability)
                            else:
                                player_dict[name]["Assisting Over 0.5 Probability"].append(0)
                            
                if "1.5" in name_text:
                    name = name_text.replace('Over 1.5', '').strip()
           
                    # Get the odds element (button following the player name)
                    odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                    odd = odd_element.get_attribute("innerText").strip()

                    # Initialize the player's dictionary if not already present
                    for p in player_dict:
                        # Prepare the player name for comparison
                        player_tokens = prepare_name(p)
                        matched_name = None
                        # Prepare the FPL player name for comparison
                        webname_tokens = prepare_name(name)

                        # Check if all tokens in one name exist in the other
                        if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                            matched_name = p
                            break

                    # Add the odds to the player's dictionary
                    if matched_name:
                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Assisting Over 1.5 Probability"].append(probability)
                        else:
                            player_dict[name]["Assisting Over 1.5 Probability"].append(0)
            
                    else:
                        for p in player_dict:
                            # Prepare the player name for comparison
                            webname_tokens = prepare_name(name)
                            matched_name = None
                            nickname1 = player_dict[p]['Nickname']
                            nickname2 = player_dict[p]['Nickname2']
                            nickname_tokens = prepare_name(nickname2)

                            if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                                matched_name = p
                                break

                        if matched_name:
                            # Calculate and add the probability
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[matched_name]["Assisting Over 1.5 Probability"].append(probability)
                            else:
                                player_dict[matched_name]["Assisting Over 1.5 Probability"].append(0)
                        else:
                            player_dict[name]['Nickname'] = 'Unknown'
                            player_dict[name]['Position'] = 'Unknown'
                            player_dict[name]['Team'] = "Unknown"
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[name]["Assisting Over 1.5 Probability"].append(probability)
                            else:
                                player_dict[name]["Assisting Over 1.5 Probability"].append(0)
                            
                if "2.5" in name_text:
                    name = name_text.replace('Over 2.5', '').strip()
           
                    # Get the odds element (button following the player name)
                    odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                    odd = odd_element.get_attribute("innerText").strip()

                    # Initialize the player's dictionary if not already present
                    for p in player_dict:
                        # Prepare the player name for comparison
                        player_tokens = prepare_name(p)
                        matched_name = None
                        # Prepare the FPL player name for comparison
                        webname_tokens = prepare_name(name)

                        # Check if all tokens in one name exist in the other
                        if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                            matched_name = p
                            break

                    # Add the odds to the player's dictionary
                    if matched_name:
                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Assisting Over 2.5 Probability"].append(probability)
                            player_dict[matched_name]["3 Assists Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Assisting Over 2.5 Probability"].append(0)
                            player_dict[matched_name]["3 Assists Probability"].append(0)
                        
                    else:
                        for p in player_dict:
                            # Prepare the player name for comparison
                            webname_tokens = prepare_name(name)
                            matched_name = None
                            nickname1 = player_dict[p]['Nickname']
                            nickname2 = player_dict[p]['Nickname2']
                            nickname_tokens = prepare_name(nickname2)

                            if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                                matched_name = p
                                break

                        if matched_name:
                            # Calculate and add the probability
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[matched_name]["Assisting Over 2.5 Probability"].append(probability)
                                player_dict[matched_name]["3 Assist Probability"].append(probability)
                            else:
                                player_dict[matched_name]["Assisting Over 2.5 Probability"].append(0)
                                player_dict[matched_name]["3 Assists Probability"].append(0)
                        else:
                            player_dict[name]['Nickname'] = 'Unknown'
                            player_dict[name]['Position'] = 'Unknown'
                            player_dict[name]['Team'] = "Unknown"
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[name]["Assisting Over 2.5 Probability"].append(probability)
                                player_dict[name]["3 Assists Probability"].append(probability)
                            else:
                                player_dict[name]["Assisting Over 2.5 Probability"].append(0)
                                player_dict[name]["3 Assists Probability"].append(0)

            except Exception as e:
                print("Could not fetch odds for a player in Player Assists")
                
        section_header.click()
        time.sleep(3)

    except Exception as e:
        print("Could not find or expand section: Player Assists")
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'Goalkeeper Saves')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand
            
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'Goalkeeper Saves')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on Goalkeeper Saves")

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'Goalkeeper Saves')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:
            try:
                name_text = player.get_attribute("innerText")
                if "2.5" in name_text:
                    name = name_text.replace('Over 2.5 Saves', '').strip()
           
                    # Get the odds element (button following the player name)
                    odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                    odd = odd_element.get_attribute("innerText").strip()

                    # Initialize the player's dictionary if not already present
                    for p in player_dict:
                        # Prepare the player name for comparison
                        player_tokens = prepare_name(p)
                        matched_name = None
                        # Prepare the FPL player name for comparison
                        webname_tokens = prepare_name(name)

                        # Check if all tokens in one name exist in the other
                        if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                            matched_name = p
                            break

                    # Add the odds to the player's dictionary
                    if matched_name:
                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Over 2.5 Saves Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Over 2.5 Saves Probability"].append(0)
                    
                    else:
                        for p in player_dict:
                            # Prepare the player name for comparison
                            webname_tokens = prepare_name(name)
                            matched_name = None
                            nickname1 = player_dict[p]['Nickname']
                            nickname2 = player_dict[p]['Nickname2']
                            nickname_tokens = prepare_name(nickname2)

                            if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                                matched_name = p
                                break

                        if matched_name:
                            # Calculate and add the probability
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[matched_name]["Over 2.5 Saves Probability"].append(probability)
                            else:
                                player_dict[matched_name]["Over 2.5 Saves Probability"].append(0)
                        else:
                            player_dict[name]['Nickname'] = 'Unknown'
                            player_dict[name]['Position'] = 'Unknown'
                            player_dict[name]['Team'] = "Unknown"
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[name]["Over 2.5 Saves Probability"].append(probability)
                            else:
                                player_dict[name]["Over 2.5 Saves Probability"].append(0)
                            
                if "5.5" in name_text:
                    name = name_text.replace('Over 5.5 Saves', '').strip()
           
                    # Get the odds element (button following the player name)
                    odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                    odd = odd_element.get_attribute("innerText").strip()

                    # Initialize the player's dictionary if not already present
                    for p in player_dict:
                        # Prepare the player name for comparison
                        player_tokens = prepare_name(p)
                        matched_name = None
                        # Prepare the FPL player name for comparison
                        webname_tokens = prepare_name(name)

                        # Check if all tokens in one name exist in the other
                        if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                            matched_name = p
                            break

                    # Add the odds to the player's dictionary
                    if matched_name:

                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Over 5.5 Saves Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Over 5.5 Saves Probability"].append(0)
                        
                    else:
                        for p in player_dict:
                            # Prepare the player name for comparison
                            webname_tokens = prepare_name(name)
                            matched_name = None
                            nickname1 = player_dict[p]['Nickname']
                            nickname2 = player_dict[p]['Nickname2']
                            nickname_tokens = prepare_name(nickname2)

                            if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                                matched_name = p
                                break

                        if matched_name:
                            # Calculate and add the probability
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[matched_name]["Over 5.5 Saves Probability"].append(probability)
                            else:
                                player_dict[matched_name]["Over 5.5 Saves Probability"].append(0)
                        else:
                            player_dict[name]['Nickname'] = 'Unknown'
                            player_dict[name]['Position'] = 'Unknown'
                            player_dict[name]['Team'] = "Unknown"
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[name]["Over 5.5 Saves Probability"].append(probability)
                            else:
                                player_dict[name]["Over 5.5 Saves Probability"].append(0)
                            
                if "8.5" in name_text:
                    name = name_text.replace('Over 8.5 Saves', '').strip()
           
                    # Get the odds element (button following the player name)
                    odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                    odd = odd_element.get_attribute("innerText").strip()

                    # Initialize the player's dictionary if not already present
                    for p in player_dict:
                        # Prepare the player name for comparison
                        player_tokens = prepare_name(p)
                        matched_name = None
                        # Prepare the FPL player name for comparison
                        webname_tokens = prepare_name(name)

                        # Check if all tokens in one name exist in the other
                        if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                            matched_name = p
                            break

                    # Add the odds to the player's dictionary
                    if matched_name:

                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Over 8.5 Saves Probability"].append(probability)
                            player_dict[matched_name]["9 Saves Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Over 8.5 Saves Probability"].append(0)
                            player_dict[matched_name]["9 Saves Probability"].append(0)
    
                    else:  
                        for p in player_dict:
                            # Prepare the player name for comparison
                            webname_tokens = prepare_name(name)
                            matched_name = None
                            nickname1 = player_dict[p]['Nickname']
                            nickname2 = player_dict[p]['Nickname2']
                            nickname_tokens = prepare_name(nickname2)

                            if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                                matched_name = p
                                break

                        if matched_name:
                            # Calculate and add the probability
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[matched_name]["Over 8.5 Saves Probability"].append(probability)
                                player_dict[matched_name]["9 Saves Probability"].append(probability)
                            else:
                                player_dict[matched_name]["Over 8.5 Saves Probability"].append(0)
                                player_dict[matched_name]["9 Saves Probability"].append(0)
                        else:
                            player_dict[name]['Nickname'] = 'Unknown'
                            player_dict[name]['Position'] = 'Unknown'
                            player_dict[name]['Team'] = "Unknown"
                            probability = 1/(float(Fraction(odd) + 1))
                            if probability is not None:
                                player_dict[name]["Over 8.5 Saves Probability"].append(probability)
                                player_dict[name]["9 Saves Probability"].append(probability)
                            else:
                                player_dict[name]["Over 8.5 Saves Probability"].append(0)
                                player_dict[name]["9 Saves Probability"].append(0)

            except Exception as e:
                print("Could not fetch odds for a player in Goalkeeper Saves")
                
        section_header.click()
        time.sleep(3)

    except Exception as e:
        print("Could not find or expand section: Goalkeeper Saves")
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'To Score A Hat-Trick')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand
            
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'To Score A Hat-Trick')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on To Score A Hat-Trick ")

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'To Score A Hat-Trick')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:
            try:
                name = player.get_attribute("innerText").strip()
                
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                for p in player_dict:
                    # Prepare the player name for comparison
                    player_tokens = prepare_name(p)
                    matched_name = None
                    # Prepare the FPL player name for comparison
                    webname_tokens = prepare_name(name)

                    # Check if all tokens in one name exist in the other
                    if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                        matched_name = p
                        break

                # Add the odds to the player's dictionary
                if matched_name:

                    # Calculate and add the probability
                    probability = 1/(float(Fraction(odd) + 1))
                    if probability is not None:
                        player_dict[matched_name]["Scoring A Hat-Trick Probability"].append(probability)
                        player_dict[matched_name]["3 Goals Probability"].append(probability)
                    else:
                        player_dict[matched_name]["Scoring A Hat-Trick Probability"].append(0)
                        player_dict[matched_name]["3 Goals Probability"].append(0)
            
                else:
                    for p in player_dict:
                        # Prepare the player name for comparison
                        webname_tokens = prepare_name(name)
                        matched_name = None
                        nickname1 = player_dict[p]['Nickname']
                        nickname2 = player_dict[p]['Nickname2']
                        nickname_tokens = prepare_name(nickname2)

                        if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                            matched_name = p
                            break

                    if matched_name:
                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Scoring A Hat-Trick Probability"].append(probability)
                            player_dict[matched_name]["3 Goals Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Scoring A Hat-Trick Probability"].append(0)
                            player_dict[matched_name]["3 Goals Probability"].append(0)
                    else:
                        player_dict[name]['Nickname'] = 'Unknown'
                        player_dict[name]['Position'] = 'Unknown'
                        player_dict[name]['Team'] = "Unknown"
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[name]["Scoring A Hat-Trick Probability"].append(probability)
                            player_dict[name]["3 Goals Probability"].append(probability)
                        else:
                            player_dict[name]["Scoring A Hat-Trick Probability"].append(0)
                            player_dict[name]["3 Goals Probabilityy"].append(0)

            except Exception as e:
                print("Could not fetch odds for a player in To Score A Hat-Trick")
                
        section_header.click()
        time.sleep(3)

    except Exception as e:
        print("Could not find or expand section: To Score A Hat-Trick")
        
    try:
        # Find the "Total Home Goals" market
        total_home_goals_header = driver.find_element(By.XPATH, "//h2[text() = 'Total Home Goals']")
        total_away_goals_header = driver.find_element(By.XPATH, "//h2[text() = 'Total Away Goals']")
        # Expand the section if it's collapsed
        if total_home_goals_header.get_attribute("aria-expanded") == "false":
            total_home_goals_header.click()
            time.sleep(3)
            
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'Total Home Goals')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on Total Home Goals ")
            
        try:
            over_05_row_home = total_home_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '0.5')]/following-sibling::button")
            away_to_concede_odds = over_05_row_home.get_attribute("innerText")
            home_0_goal = 1 - (1/(float(Fraction(away_to_concede_odds)) + 1))
        except NoSuchElementException:
            home_0_goal = 0
            
        try:
            over_15_row_home = total_home_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '1.5')]/following-sibling::button")
            home_over_15 = over_15_row_home.get_attribute("innerText")
            home_1_goal = (1 - (1/(float(Fraction(home_over_15)) + 1))) - home_0_goal
            home_2_goal = (1/(float(Fraction(home_over_15)) + 1))
        except NoSuchElementException:
            home_1_goal = 0
            home_2_goal = 0

        try:
            over_25_row_home = total_home_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '2.5')]/following-sibling::button")
            home_over_25 = over_25_row_home.get_attribute("innerText")
            home_2_goal = (1 - (1/(float(Fraction(home_over_25)) + 1))) - home_1_goal - home_0_goal
            home_3_goal = (1/(float(Fraction(home_over_25)) + 1))
        except NoSuchElementException:
            home_3_goal = 0

        try:
            over_35_row_home = total_home_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '3.5')]/following-sibling::button")
            home_over_35 = over_35_row_home.get_attribute("innerText")
            home_3_goal = (1 - (1/(float(Fraction(home_over_35)) + 1))) - home_2_goal - home_1_goal - home_0_goal
            home_4_goal = (1/(float(Fraction(home_over_35)) + 1))
        except NoSuchElementException:
            try:
                under_35_row_home = total_home_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[1]//p[contains(text(), '3.5')]/following-sibling::button")
                home_under_35 = under_35_row_home.get_attribute("innerText")
                home_3_goal = (1/(float(Fraction(home_under_35)) + 1)) - home_2_goal - home_1_goal - home_0_goal
                home_4_goal = 1 - (1/(float(Fraction(home_under_35)) + 1))
            except NoSuchElementException:
                home_4_goal = 0
            
        total_home_goals_header.click()
        time.sleep(3)
        
        if total_away_goals_header.get_attribute("aria-expanded") == "false":
            total_away_goals_header.click()
            time.sleep(3)
        
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'Total Away Goals')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on Total Away Goals ")
            
        try:
            over_05_row_away = total_away_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '0.5')]/following-sibling::button")
            home_to_concede_odds = over_05_row_away.get_attribute("innerText")
            away_0_goal = 1 - (1/(float(Fraction(home_to_concede_odds)) + 1))
        except NoSuchElementException:
            away_0_goal = 0
            
        try:
            over_15_row_away = total_away_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '1.5')]/following-sibling::button")
            away_over_15 = over_15_row_away.get_attribute("innerText")
            away_1_goal = (1 - (1/(float(Fraction(away_over_15)) + 1))) - away_0_goal
            away_2_goal = (1/(float(Fraction(away_over_15)) + 1))
        except NoSuchElementException:
            away_1_goal = 0
            away_2_goal = 0
        try:
            over_25_row_away = total_away_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '2.5')]/following-sibling::button")
            away_over_25 = over_25_row_away.get_attribute("innerText")
            away_2_goal = (1 - (1/(float(Fraction(away_over_25)) + 1))) - away_1_goal - away_0_goal
            away_3_goal = (1/(float(Fraction(away_over_25)) + 1))
        except NoSuchElementException:
            away_3_goal = 0
        try:
            over_35_row_away = total_away_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[2]//p[contains(text(), '3.5')]/following-sibling::button")
            away_over_35 = over_35_row_away.get_attribute("innerText")
            away_3_goal = (1 - (1/(float(Fraction(away_over_35)) + 1))) - away_2_goal - away_1_goal - away_0_goal
            away_4_goal = (1/(float(Fraction(away_over_35)) + 1))
        
        except NoSuchElementException:
            try:
                under_35_row_away = total_away_goals_header.find_element(By.XPATH, "./following-sibling::*[1]/*[1]/*[3]/*[1]//p[contains(text(), '3.5')]/following-sibling::button")
                away_under_35 = under_35_row_away.get_attribute("innerText")
                away_3_goal = (1/(float(Fraction(away_under_35)) + 1)) - away_2_goal - away_1_goal - away_0_goal
                away_4_goal = 1 - (1/(float(Fraction(away_under_35)) + 1))
            except NoSuchElementException:
                away_4_goal = 0
            
        total_away_goals_header.click()
        time.sleep(3)
        
    except Exception as e:
        print("Couldnt find odds for Goals Scored")

    for player in player_dict:
        if (player_dict[player]['Team'] == home_team) and (player_dict[player]['Position'] in ['MNG', 'GKP', 'DEF', 'MID']):
            player_dict[player]['Clean Sheet Probability'].append(away_0_goal)
            if player_dict[player]['Position'] == 'MNG':
                player_dict[player]['Team Scores 0 Goals Probability'].append(home_0_goal)
                player_dict[player]['Team Scores 1 Goal Probability'].append(home_1_goal)
                player_dict[player]['Team Scores 2 Goals Probability'].append(home_2_goal)
                player_dict[player]['Team Scores 3 Goals Probability'].append(home_3_goal)
                player_dict[player]['Team Scores 4 Goals Probability'].append(home_4_goal)
        if (player_dict[player]['Team'] == away_team) and (player_dict[player]['Position'] in ['MNG', 'GKP', 'DEF', 'MID']):
            player_dict[player]['Clean Sheet Probability'].append(home_0_goal)
            if player_dict[player]['Position'] == 'MNG':
                player_dict[player]['Team Scores 0 Goals Probability'].append(away_0_goal)
                player_dict[player]['Team Scores 1 Goal Probability'].append(away_1_goal)
                player_dict[player]['Team Scores 2 Goals Probability'].append(away_2_goal)
                player_dict[player]['Team Scores 3 Goals Probability'].append(away_3_goal)
                player_dict[player]['Team Scores 4 Goals Probability'].append(away_4_goal)
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'Anytime Goalscorer')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand
        
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'Anytime Goalscorer')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on Anytime Goalscorer ")

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'Anytime Goalscorer')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:
            try:
                name = player.get_attribute("innerText").strip()
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                for p in player_dict:
                    # Prepare the player name for comparison
                    player_tokens = prepare_name(p)
                    matched_name = None
                    # Prepare the FPL player name for comparison
                    webname_tokens = prepare_name(name)

                    # Check if all tokens in one name exist in the other
                    if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                        matched_name = p
                        break

                # Add the odds to the player's dictionary
                if matched_name:

                    # Calculate and add the probability
                    probability = 1/(float(Fraction(odd) + 1))
                    if probability is not None:
                        player_dict[matched_name]["Scoring Anytime Probability"].append(probability)
                    else:
                        player_dict[matched_name]["Scoring Anytime Probability"].append(0)
    
                else:
                    for p in player_dict:
                        # Prepare the player name for comparison
                        webname_tokens = prepare_name(name)
                        matched_name = None
                        nickname1 = player_dict[p]['Nickname']
                        nickname2 = player_dict[p]['Nickname2']
                        nickname_tokens = prepare_name(nickname2)

                        if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                            matched_name = p
                            break

                    if matched_name:
                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Scoring Anytime Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Scoring Anytime Probability"].append(0)
                    else:
                        player_dict[name]['Nickname'] = 'Unknown'
                        player_dict[name]['Position'] = 'Unknown'
                        player_dict[name]['Team'] = "Unknown"
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[name]["Scoring Anytime Probability"].append(probability)
                        else:
                            player_dict[name]["Scoring Anytime Probability"].append(0)

            except Exception as e:
                print("Could not fetch odds for a player in Anytime Goalscorer")
                
        section_header.click()
        time.sleep(3)

    except Exception as e:
        print("Could not find or expand section: Anytime Goalscorer")
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'To Score 2 Or More Goals')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand
            
        wait = WebDriverWait(driver, 3)
        try:
            show_more = wait.until(EC.element_to_be_clickable((By.XPATH, f"//h2[contains(text(), 'To Score 2 Or More Goals')]/following-sibling::*[1]/*[1]//div[@class='ShowMoreContainer_s1ts54as']/span[@class='ShowMoreText_sjatvm6']")))
            show_more.click()
            time.sleep(2)
        except Exception as e:
            print("Could not click Show More on To Score 2 Or More Goals ")

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'To Score 2 Or More Goals')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:
            try:
                name = player.get_attribute("innerText").strip()
        
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                for p in player_dict:
                    # Prepare the player name for comparison
                    player_tokens = prepare_name(p)
                    matched_name = None
                    # Prepare the FPL player name for comparison
                    webname_tokens = prepare_name(name)

                    # Check if all tokens in one name exist in the other
                    if all(token in webname_tokens for token in player_tokens) or all(token in player_tokens for token in webname_tokens):
                        matched_name = p
                        break

                # Add the odds to the player's dictionary
                if matched_name:

                    # Calculate and add the probability
                    probability = 1/(float(Fraction(odd) + 1))
                    if probability is not None:
                        player_dict[matched_name]["Scoring 2 Or More Goals Probability"].append(probability)
                    else:
                        player_dict[matched_name]["Scoring 2 Or More Goals Probability"].append(0)
            
                else:
                    for p in player_dict:
                        # Prepare the player name for comparison
                        webname_tokens = prepare_name(name)
                        matched_name = None
                        nickname1 = player_dict[p]['Nickname']
                        nickname2 = player_dict[p]['Nickname2']
                        nickname_tokens = prepare_name(nickname2)

                        if nickname1 in name or all(token in nickname_tokens for token in webname_tokens) or all(token in webname_tokens for token in nickname_tokens):
                            matched_name = p
                            break

                    if matched_name:
                        # Calculate and add the probability
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[matched_name]["Scoring 2 Or More Goals Probability"].append(probability)
                        else:
                            player_dict[matched_name]["Scoring 2 Or More Goals Probability"].append(0)
                    else:
                        player_dict[name]['Nickname'] = 'Unknown'
                        player_dict[name]['Position'] = 'Unknown'
                        player_dict[name]['Team'] = "Unknown"
                        probability = 1/(float(Fraction(odd) + 1))
                        if probability is not None:
                            player_dict[name]["Scoring 2 Or More Goals Probability"].append(probability)
                        else:
                            player_dict[name]["Scoring 2 Or More Goals Probability"].append(0)

            except Exception as e:
                print("Could not fetch odds for a player in To Score 2 Or More Goals")

    except Exception as e:
        print("Could not find or expand section: To Score 2 Or More Goals")

driver.quit()

for player, odds in player_dict.items():
    if odds.get("Position") in ['DEF', 'MID', 'FWD', 'Unknown']:
        anytime_prob = odds.get("Scoring Anytime Probability", [])
        two_or_more_prob = odds.get("Scoring 2 Or More Goals Probability", [])
        hattrick_prob = odds.get("Scoring A Hat-Trick Probability", [])
        assisting_over_05_prob = odds.get("Assisting Over 0.5 Probability", [])
        assisting_over_15_prob = odds.get("Assisting Over 1.5 Probability", [])
        assisting_over_25_prob = odds.get("Assisting Over 2.5 Probability", [])

        for p25, p15, p05 in zip_longest(assisting_over_25_prob, assisting_over_15_prob, assisting_over_05_prob, fillvalue=0):
            player_dict[player]["2 Assists Probability"].append(max((p15 - p25), 0))
            player_dict[player]["1 Assist Probability"].append(max((p05 - p15), 0))

        for p3, p2, p1 in zip_longest(hattrick_prob, two_or_more_prob, anytime_prob, fillvalue=0):
            player_dict[player]["2 Goals Probability"].append(max((p2 - p3), 0))
            player_dict[player]["1 Goal Probability"].append(max((p1 - p2), 0))
            
    if odds.get("Position") == 'GKP':
        over_25_saves = odds.get("Over 2.5 Saves Probability", [])
        over_55_saves = odds.get("Over 5.5 Saves Probability", [])
        over_85_saves = odds.get("Over 8.5 Saves Probability", [])

        for s85, s55, s25 in zip_longest(over_85_saves, over_55_saves, over_25_saves, fillvalue=0):
            player_dict[player]["6 Saves Probability"].append(max((s55 - s85), 0))
            player_dict[player]["3 Saves Probability"].append(max((s25 - s55), 0))

"""
Calculates the "Points" column for each player based on the provided formula.
"""
for player, odds in player_dict.items():
    try:
        # Get probabilities
        team = odds.get("Team", "Unknown")
        number_of_games = teams_playing[team] if team != 'Unknown' else 1
        one_goal = odds.get("1 Goal Probability", [])
        two_goals = odds.get("2 Goals Probability", [])
        three_goals = odds.get("3 Goals Probability", [])
        one_assist = odds.get("1 Assist Probability", [])
        two_assists = odds.get("2 Assists Probability", [])
        three_assists = odds.get("3 Assists Probability", [])          
        cs_odd = odds.get("Clean Sheet Probability", [])
        position = odds.get("Position", "Unknown")
        three_saves = odds.get("3 Saves Probability", [])
        six_saves = odds.get("6 Saves Probability", [])
        nine_saves = odds.get("9 Saves Probability", [])
        one_goal_scored = odds.get("Team Scores 1 Goal Probability", [])
        two_goals_scored = odds.get("Team Scores 2 Goals Probability", [])
        three_goals_scored = odds.get("Team Scores 3 Goals Probability", [])
        four_goals_scored = odds.get("Team Scores 4 Goals Probability", [])

        win_probability =  odds.get('Win Probability', [])
        draw_probability =  odds.get('Draw Probability', [])
        MGR_Bonus = odds.get('Manager Bonus', [])
        chance_of_playing = odds.get("Chance of Playing", 1)
             
        # Calculate points
        if position in ('MID'):
            points = chance_of_playing * (
            number_of_games * 
            2 + 3 * sum(three_goals) * 5 +
            2 * sum(two_goals) * 5 +
            sum(one_goal) * 5 +
            3 * sum(three_assists) * 3 +
            2 * sum(two_assists) * 3 +
            sum(one_assist) * 3 +
            sum(cs_odd))
        if position in ('DEF'):
            points = chance_of_playing * (
            number_of_games * 
            2 + 3 * sum(three_goals) * 6 +
            2 * sum(two_goals) * 6 +
            sum(one_goal) * 6 +
            3 * sum(three_assists) * 3 +
            2 * sum(two_assists) * 3 +
            sum(one_assist) * 3 +
            sum(cs_odd) * 4)
        if position in ('GKP'):
            points = chance_of_playing * (
            number_of_games * 
            2 + 3 * sum(nine_saves) +
            2 * sum(six_saves) +
            sum(three_saves) +
            sum(cs_odd) * 4)
        if position in ('FWD'):
            points = chance_of_playing * (
            number_of_games * 
            2 + 3 * sum(three_goals) * 4 +
            2 * sum(two_goals) * 4 +
            sum(one_goal) * 4 +
            3 * sum(three_assists) * 3 +
            2 * sum(two_assists) * 3 +
            sum(one_assist) * 3)
        if position in ('Unknown'):
            points = chance_of_playing * (
            number_of_games * 
            2 + 3 * sum(three_goals) * 4 +
            2 * sum(two_goals) * 4 +
            sum(one_goal) * 4 +
            3 * sum(three_assists) * 3 +
            2 * sum(two_assists) * 3 +
            sum(one_assist) * 3)
        if position in ('MNG'):
            points = 0
            if len(win_probability) > 0:
                for w, d, b in zip_longest(win_probability, draw_probability, MGR_Bonus, fillvalue=0):
                    points += w * 6 + d * 3
                    if b == 'True':
                        points += w * 10 + d * 5
                points += sum(cs_odd) * 2 + sum(one_goal_scored) + 2 * sum(two_goals_scored) + 3 * sum(three_goals_scored) + 4 * max(sum(four_goals_scored), 0)

        player_dict[player]['Points'] = round(points, 3)
    except Exception as e:
        print(f"Could not calculate points for {player}: {e}")
        

Prompt for accessing outside UK did not pop up
Prompt for accepting Cookies did not pop up
Ad did not pop up
Could not find or expand section: Player Assists
Could not find or expand section: Goalkeeper Saves
Prompt for accessing outside UK did not pop up
Prompt for accepting Cookies did not pop up
Ad did not pop up
Could not find or expand section: Player Assists
Could not find or expand section: Goalkeeper Saves
Prompt for accessing outside UK did not pop up
Prompt for accepting Cookies did not pop up
Ad did not pop up
Could not find or expand section: Player Assists
Could not find or expand section: Goalkeeper Saves
Prompt for accessing outside UK did not pop up
Prompt for accepting Cookies did not pop up
Could not find or expand section: Player Assists
Could not find or expand section: Goalkeeper Saves
Prompt for accessing outside UK did not pop up
Prompt for accepting Cookies did not pop up
Ad did not pop up
Could not find or expand section: Player Assists
Could not find or expand

In [ ]:
player_data_df = pd.DataFrame.from_dict(player_dict, orient='index')
player_data_df.index.name = 'Player'

player_points_df = player_data_df[['Position', 'Team', 'Points']]
sorted_player_points_df = player_points_df.sort_values(by=['Points'], ascending=False)

print(sorted_player_points_df.head(20))

# Create outputs directory if it doesn't exist
if not os.path.exists("outputs"):
    os.makedirs("outputs")
with pd.ExcelWriter(f"outputs/gw_{next_gameweek}_output.xlsx") as writer:
    player_data_df.to_excel(writer, sheet_name='Data')
    sorted_player_points_df.to_excel(writer, sheet_name='Points')


                                       Position               Team  Points
Player                                                                    
Kieran McKenna                              MNG            Ipswich   8.132
Ruud van Nistelrooij                        MNG          Leicester   7.073
Pep Guardiola                               MNG           Man City   6.864
Thomas Frank                                MNG          Brentford   6.830
Andoni Iraola                               MNG        Bournemouth   6.636
Mikel Arteta                                MNG            Arsenal   6.495
Ruben Filipe Marques Diogo Amorim           MNG            Man Utd   6.484
Marco Alexandre Saraiva da Silva            MNG             Fulham   6.108
Nuno Herlander Simões Espírito Santo        MNG  Nottingham Forest   5.917
Vítor Manuel de Oliveira Lopes Pereira      MNG      Wolverhampton   5.860
David Moyes                                 MNG            Everton   5.789
Martial Godo             